In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
dataFrame = pd.read_csv('22.txt', sep=" ", header=None, skiprows=1, names=['w', 'v', 'p'])
WVVals = pd.read_csv('22.txt', sep=" ", header=None, nrows=1)

maxWeight = WVVals.iloc[0][0]
maxVolume = WVVals.iloc[0][1]

weightList = dataFrame['w'].values.tolist()
volumeList = dataFrame['v'].values.tolist()
priceList = dataFrame['p'].values.tolist()

In [3]:
def fitness(currSet):
    currWeight = sum([a * b for a, b in zip(weightList, currSet)])
    currVolume = sum([a * b for a, b in zip(volumeList, currSet)])
    currPrice = sum([a * b for a, b in zip(priceList, currSet)])

    if currVolume > maxVolume or currWeight > maxWeight:
        return 0;

    return currPrice;


In [4]:
# Шаг 1 . Случайная генерация
def generateEntity(length):
    return [random.randint(0, 1) for i in range(length)]

def generation(length, entities):
    sets = []
    setID=0;
    for i in range(entities):
        newSet = generateEntity(length)
        sets.append([setID, newSet, fitness(newSet)])
        setID=setID+1
    return pd.DataFrame(sets,columns=['id', 'set', 'fitness'])




In [5]:
# Шаг 2. Массовая игра в русскую рулетку
def rouleteSelection(setsDF):
    setsData = setsDF[['id','set','fitness']]
    setsData = setsData[setsData['fitness'] != 0]
    totalFitness = sum(setsData['fitness'].values.tolist())
    maxSectVal = 1;

    setsData['section'] = setsData.apply(lambda row: row['fitness']/totalFitness, axis=1)
    setsData = setsData.sort_values(by=['fitness'],ascending=False)

    sectionsFixed = setsData['section'].values.tolist()
    newSections = []
    for fit in sectionsFixed:
        maxSectVal = maxSectVal-fit
        if(maxSectVal<0):
            maxSectVal = 0
        fit = maxSectVal
        newSections.append(maxSectVal)
    setsData['section'] = newSections
    return setsData[setsData['section']>random.random()][['id','set','fitness']]

In [6]:
#Создание пар
def createPairs(selectedSets):
    totalPairsCount = len(selectedSets[set].values.tolist())
    selectedSets['pair'] = [0 for i in range(totalPairsCount)]
    currPairNum = 0

    while True:
        pairless = selectedSets[selectedSets['pair']==0]
        pairlessCount = len(pairless.values.tolist())
    
        if(pairlessCount < 2):
            break
        
        while True:
            firstParentIdx = random.randint(0,pairlessCount-1)
            secondParentIdx = random.randint(0,pairlessCount-1)
            if(firstParentIdx != secondParentIdx):
                break
        selectedSets.iloc[[firstParentIdx,secondParentIdx],3] = currPairNum
        selectedSets = selectedSets.sort_values(by=['pair'])
        currPairNum=currPairNum+1
    return selectedSets

#Шаг 5. Формирование новой популяции (замена своих родителей)

#Шаг 6. Оценка

In [7]:
#Шаг 3. Кроссинговер (однородный (каждый бит от случайно выбранного родителя))
def breed(setPair):
    return [setPair[random.randint(0,1)][i] for i in range(0,len(setPair[0]))]

In [8]:
#Шаг 4. Мутация (инвертирование всех битов у 1 особи )
def mutation(setPair):
    randnum = random.randint(0,1)
    return [(~np.array(setPair[randnum]).astype('bool')).astype('int').tolist(),setPair[1-randnum]]

In [9]:
def newPop(setsPairs):
    columnNames = ['id', 'set', 'fitness']
    pairsCount = setsPairs["pair"].max()
    newDF = pd.DataFrame(columns=columnNames)
    for pairIDX in range(1,pairsCount+1):
        setPair = setsPairs[setsPairs['pair'] == pairIDX]['set'].values.tolist()
        children = [breed(setPair),breed(setPair)]
        children = mutation(children)
        childIDS = setsPairs[setsPairs['pair'] == pairIDX]['id'].values.tolist()
        childID = 0
        for child in children:
            childDF = pd.DataFrame([[childIDS[childID],child,fitness(child)]],columns=columnNames)
            childID = childID+1
            newDF = newDF.append(childDF)
    return newDF

In [10]:
setsDF = generation(5, 200)
setsDF

,id,set,fitness
0,0,"[0, 0, 1, 0, 1]",527
1,1,"[1, 0, 0, 1, 0]",533
2,2,"[0, 1, 0, 1, 1]",824
3,3,"[0, 1, 1, 0, 0]",358
4,4,"[0, 0, 0, 0, 0]",0
...,...,...,...
195,195,"[1, 1, 0, 1, 1]",1056
196,196,"[1, 0, 0, 0, 1]",578
197,197,"[0, 1, 1, 0, 1]",704
198,198,"[1, 0, 1, 1, 0]",714


In [11]:
selectedSets = rouleteSelection(setsDF)
selectedSets

,id,set,fitness
83,83,"[1, 1, 1, 1, 1]",1237
179,179,"[1, 1, 1, 1, 1]",1237
173,173,"[1, 1, 1, 1, 1]",1237
13,13,"[1, 0, 1, 1, 1]",1060
171,171,"[1, 0, 1, 1, 1]",1060
...,...,...,...
153,153,"[1, 1, 1, 0, 0]",590
15,15,"[1, 1, 1, 0, 0]",590
39,39,"[1, 1, 1, 0, 0]",590
88,88,"[1, 1, 1, 0, 0]",590


In [12]:
setsPairs = createPairs(selectedSets)
setsPairs

,id,set,fitness,pair
148,148,"[1, 0, 1, 1, 0]",714,0
169,169,"[1, 0, 0, 1, 1]",879,1
109,109,"[1, 1, 0, 1, 0]",710,1
23,23,"[1, 1, 0, 0, 1]",755,2
78,78,"[1, 0, 1, 1, 0]",714,2
...,...,...,...,...
94,94,"[0, 1, 1, 1, 1]",1005,47
69,69,"[1, 0, 1, 0, 1]",759,48
136,136,"[1, 1, 1, 0, 0]",590,48
150,150,"[1, 1, 1, 0, 1]",936,49


In [13]:
newSets = newPop(setsPairs[setsPairs['pair']!=0])
newSets

,id,set,fitness
0,169,"[0, 0, 1, 0, 0]",181
0,109,"[1, 1, 0, 1, 0]",710
0,23,"[0, 1, 0, 1, 1]",824
0,78,"[1, 1, 1, 0, 1]",936
0,8,"[1, 1, 1, 1, 1]",1237
...,...,...,...
0,94,"[0, 1, 1, 1, 1]",1005
0,69,"[0, 0, 0, 1, 0]",301
0,136,"[1, 0, 1, 0, 0]",413
0,150,"[1, 0, 0, 1, 0]",533


In [14]:
newIDS = newSets['id'].values.tolist()
setsDF = setsDF[~setsDF['id'].isin(newIDS)].append(newSets)

In [15]:
setsDF

,id,set,fitness
0,0,"[0, 0, 1, 0, 1]",527
1,1,"[1, 0, 0, 1, 0]",533
3,3,"[0, 1, 1, 0, 0]",358
4,4,"[0, 0, 0, 0, 0]",0
5,5,"[0, 0, 1, 0, 1]",527
...,...,...,...
0,94,"[0, 1, 1, 1, 1]",1005
0,69,"[0, 0, 0, 1, 0]",301
0,136,"[1, 0, 1, 0, 0]",413
0,150,"[1, 0, 0, 1, 0]",533


In [16]:
iterationCount=0
setsDF = generation(len(priceList), 200)
while(iterationCount<500):
    prevmaxfit=setsDF['fitness'].max()    
    selectedSets = rouleteSelection(setsDF)
    setsPairs = createPairs(selectedSets)
    if (len(setsPairs[setsPairs['pair']!=0].values.tolist()) == 0):
        continue
    newSets = newPop(setsPairs[setsPairs['pair']!=0])
    newIDS = newSets['id'].values.tolist()
    setsDF = setsDF[~setsDF['id'].isin(newIDS)].append(newSets)
    iterationCount= iterationCount+1
    if(abs(setsDF['fitness'].max()-prevmaxfit)<min(priceList)):
        break
setsDF['fitness'].max()

2898